In [ ]:
"""
Week 4: Monitoring and Error Handling System for NewsAI
Robust error handling, logging, and performance monitoring
"""

import sys
sys.path.append('../..')

import logging
import json
import time
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Optional, Any
import pandas as pd
from dataclasses import dataclass, asdict
from collections import defaultdict, deque
import traceback
import threading
import psutil
import warnings

from config.settings import *

@dataclass
class PerformanceMetrics:
    """Structure for tracking performance metrics"""
    timestamp: str
    articles_processed: int
    processing_time: float
    articles_per_second: float
    memory_usage_mb: float
    cpu_usage_percent: float
    error_rate: float
    avg_confidence: float
    approval_rate: float

@dataclass
class ErrorEvent:
    """Structure for tracking errors"""
    timestamp: str
    error_type: str
    error_message: str
    article_id: str
    stack_trace: str
    recovery_action: str
    severity: str  # 'low', 'medium', 'high', 'critical'

class NewsAIMonitor:
    """Comprehensive monitoring system for NewsAI"""
    
    def __init__(self, log_dir: Path = None, max_metrics_history: int = 1000):
        self.log_dir = log_dir or ANALYTICS_DIR
        self.log_dir.mkdir(exist_ok=True)
        
        # Metrics tracking
        self.metrics_history = deque(maxlen=max_metrics_history)
        self.error_history = deque(maxlen=max_metrics_history)
        self.performance_data = defaultdict(list)
        
        # Real-time counters
        self.session_stats = {
            'start_time': time.time(),
            'articles_processed': 0,
            'successful_predictions': 0,
            'errors': 0,
            'total_processing_time': 0.0,
            'confidence_scores': [],
            'approval_decisions': []
        }
        
        # Thread safety
        self.lock = threading.Lock()
        
        # Setup logging
        self.setup_logging()
        
        # Setup alerts
        self.alert_thresholds = {
            'error_rate': 0.1,  # 10% error rate
            'low_confidence': 0.3,  # Confidence below 30%
            'processing_time': 10.0,  # Processing time > 10 seconds
            'memory_usage': 1000,  # Memory usage > 1GB
            'cpu_usage': 80  # CPU usage > 80%
        }
        
        self.logger.info("NewsAI Monitor initialized")
    
    def setup_logging(self):
        """Setup comprehensive logging system"""
        
        # Create formatters
        detailed_formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        )
        
        simple_formatter = logging.Formatter(
            '%(asctime)s - %(levelname)s - %(message)s'
        )
        
        # Setup main logger
        self.logger = logging.getLogger('NewsAI')
        self.logger.setLevel(logging.INFO)
        
        # File handlers
        log_file = self.log_dir / 'newsai.log'
        file_handler = logging.FileHandler(log_file, encoding='utf-8')
        file_handler.setFormatter(detaile